In [11]:
import re
import pandas as pd
import numpy as np
import pickle

from sklearn.impute import KNNImputer

import seaborn as sns

from sklearn.decomposition import NMF

# Import movie data

In [12]:
path = 'recommender/data/ml-latest-small/ratings.csv'
df_rating = pd.read_csv(path, sep=',')
df_rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [13]:
path1 = 'recommender/data/ml-latest-small/movies.csv'
df_movies = pd.read_csv(path1, sep=',')
df_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [14]:
df_sp = df_movies.copy()
df_sp['just the title'] = df_sp['title'].copy
for i in range(len(df_movies.index)):
    title = df_sp['title'][i]
    title = re.sub(' \([0-9]{4}\)', "", title)
    df_sp['just the title'][i] = title
df_sp

/Users/anja.jesierski/opt/anaconda3/envs/dl/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,movieId,title,genres,just the title
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,No Game No Life: Zero
9739,193585,Flint (2017),Drama,Flint
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,Bungo Stray Dogs: Dead Apple


In [15]:
df_movies['title only'] = df_sp['just the title']
df_movies.head()

,movieId,title,genres,title only
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II


In [16]:
df_movies.loc[df_movies['title only'] == 'Sabrina']

,movieId,title,genres,title only
6,7,Sabrina (1995),Comedy|Romance,Sabrina
697,915,Sabrina (1954),Comedy|Romance,Sabrina


# movie, ratings by users

In [17]:
df_rating.groupby('movieId')['rating'].mean().sort_values(ascending=False).head(10)

movieId
53355     5.0
96608     5.0
33138     5.0
128087    5.0
3851      5.0
160644    5.0
126921    5.0
3939      5.0
3940      5.0
3941      5.0
Name: rating, dtype: float64

In [18]:
# movie, ratings by users
R = df_rating.set_index(['movieId', 'userId'])['rating'].unstack(0)

In [19]:
type(R)

pandas.core.frame.DataFrame

In [20]:
R

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
#make R.column labels movie titles
R_movieIds = pd.DataFrame(R.columns)
titles = R_movieIds.merge(df_movies, on='movieId')
R.columns = titles['title']
R

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
imputer = KNNImputer() # k-nearste neighor

In [23]:
R_imputed = pd.DataFrame(imputer.fit_transform(R), index=R.index, columns=R.columns)
R_imputed

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,3.6,4.0,2.8,3.4,4.0,3.2,3.0,2.9,3.6,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
2,4.0,3.6,2.9,2.1,2.2,3.7,2.8,3.0,2.4,3.3,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
3,3.7,3.0,2.1,2.7,2.8,3.3,2.5,2.8,2.8,4.0,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
4,3.9,3.6,3.6,2.5,3.2,3.2,3.6,3.0,3.4,3.7,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
5,4.0,3.4,3.3,2.1,2.6,4.0,3.2,2.6,3.5,3.5,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,3.6,3.3,2.7,2.8,3.8,2.5,2.4,2.6,3.9,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
607,4.0,3.8,4.0,2.8,3.6,3.8,4.0,2.6,3.3,3.9,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0
608,2.5,2.0,2.0,2.1,2.9,4.3,3.2,2.8,2.6,4.0,...,3.5,3.0,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0


### save imputed data frame R to speed up web app

In [24]:
R_imputed.to_pickle('recommender/data/R_imputed.pkl')

In [25]:
average_movie_rating = R.mean().mean()
average_movie_rating#used later for new user fill nan
#average_movie_rating

3.262448274810963

# Train NMF

In [26]:
# Instantiate the nmf
nmf = NMF(n_components=20, max_iter=10000)

In [28]:
# As usual with scikit-learn Classes, we fit the nmf
nmf.fit(R_imputed)

/Users/anja.jesierski/opt/anaconda3/envs/dl/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 10000 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


NMF(max_iter=10000, n_components=20)

In [29]:
R_imputed.shape

(610, 9724)

### save model for later use in other files

In [32]:
filepath = 'recommender/data/recommender_model_n-comp-20_max-iter-10000_KNN-imputer.sav'

In [33]:
with open(filepath, 'wb') as file:
    pickle.dump(nmf, file)

## Check out the sub-matrices and reconstruction error

In [35]:
# Extract the movie-feature matrix
Q = nmf.components_
Q.shape # shape of Q should have a shape of? Anja: 20 features (rows) x # movies = 9724 (columns) 

(20, 9724)

In [36]:
# Make a data frame
pd.DataFrame(Q, columns=R.columns)

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
0,0.554892,0.000000,0.000000,0.396088,0.000000,0.399605,0.395390,0.352223,0.000000,0.031554,...,1.037006,0.888862,1.185149,1.185149,1.037006,1.185149,1.037006,1.037006,1.037006,1.185149
1,11.705885,3.905192,8.308843,7.445097,13.164491,14.195430,14.216777,3.268387,3.182182,2.240874,...,10.776849,9.237299,12.316398,12.316398,10.776849,12.316398,10.776849,10.776849,10.776849,12.316398
2,3.711294,4.237563,4.265421,1.903132,3.531747,1.681817,1.957517,2.614583,4.137542,2.298777,...,0.563540,0.483034,0.644046,0.644046,0.563540,0.644046,0.563540,0.563540,0.563540,0.644046
3,0.000000,1.317693,1.206127,0.780156,0.554057,1.339850,0.215621,1.389000,1.020936,2.868640,...,0.820341,0.703150,0.937533,0.937533,0.820341,0.937533,0.820341,0.820341,0.820341,0.937533
4,1.646239,0.658106,1.781446,0.747340,0.896404,1.469380,0.846054,0.402881,1.419980,1.121189,...,0.544907,0.467063,0.622751,0.622751,0.544907,0.622751,0.544907,0.544907,0.544907,0.622751
5,0.242973,0.269134,0.241930,0.140250,0.095805,0.302231,0.086553,0.160548,0.034887,0.383011,...,0.106093,0.090937,0.121249,0.121249,0.106093,0.121249,0.106093,0.106093,0.106093,0.121249
6,1.513014,2.042737,1.077190,0.337347,1.041522,2.007562,0.603034,0.525821,0.977467,1.974463,...,0.250905,0.215062,0.286749,0.286749,0.250905,0.286749,0.250905,0.250905,0.250905,0.286749
7,0.513825,0.278764,0.317734,0.071052,0.395077,0.261875,0.287544,0.000000,0.342017,0.361509,...,0.125566,0.107628,0.143503,0.143503,0.125566,0.143503,0.125566,0.125566,0.125566,0.143503
8,0.069507,0.645241,1.569598,0.580858,1.223676,1.251690,0.878941,0.597877,1.527026,1.412237,...,0.129253,0.110788,0.147718,0.147718,0.129253,0.147718,0.129253,0.129253,0.129253,0.147718
9,0.337328,0.332109,0.176344,0.023929,0.208099,0.200428,0.019488,0.105439,0.187467,0.427596,...,0.041005,0.035147,0.046863,0.046863,0.041005,0.046863,0.041005,0.041005,0.041005,0.046863


In [37]:
# Extract the user-feature matrix
P = nmf.transform(R_imputed)
P.shape # We expect a shape of? # userId 610 (rows) x 20 components (columns)

(610, 20)

In [38]:
# Make a DataFrame out of it
pd.DataFrame(P, index=R.index)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
userId,,,,,,,,,,,,,,,,,,,,
1,2.273584,0.046313,0.151323,0.101013,0.178409,0.882119,0.006210,0.218067,0.210082,1.065397,0.076692,0.056284,0.468171,0.477577,0.254814,1.124785,0.647839,0.098888,0.332116,0.153176
2,2.534618,0.019207,0.060413,0.161997,0.113888,0.892146,0.107364,0.629494,0.000000,0.395359,0.131367,0.087137,0.221113,0.686561,0.204030,1.913765,0.445706,0.038667,0.576105,0.564450
3,2.634766,0.013536,0.000000,0.145892,0.000000,1.546588,0.189364,0.765902,0.127557,0.563431,0.028581,0.000000,0.000000,0.000000,0.000000,2.358946,0.000000,0.145634,1.620786,0.000000
4,2.801246,0.000000,0.116130,0.000000,0.244076,0.105524,0.003756,0.923961,0.000000,0.764318,0.157554,0.464077,0.331026,0.355963,0.346442,2.056841,0.000000,0.075084,0.707629,0.000000
5,2.752124,0.008263,0.088523,0.032023,0.111091,0.570296,0.130991,0.624177,0.089017,0.571523,0.139682,0.000000,0.018792,0.354098,0.260252,0.665961,1.280468,0.180971,0.427672,0.491056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.793894,0.002683,0.010034,0.041729,0.240739,0.008164,0.180792,0.296967,0.000000,0.464515,0.359624,0.346946,0.341174,0.180388,0.088275,1.366046,1.138298,0.089956,0.273218,0.856848
607,2.516473,0.025516,0.124082,0.052566,0.158467,0.887386,0.009051,0.187658,0.213710,0.000000,0.210300,0.376697,0.382440,1.042992,0.064343,1.796466,1.104145,0.028314,0.000000,0.407536
608,2.727672,0.005825,0.000000,0.357792,0.002356,0.000000,0.340625,0.094628,0.343640,0.209900,0.030344,0.487088,0.000000,0.565891,0.000000,0.769933,0.507057,0.088124,0.000000,0.528377


In [39]:
# Look at the reconstruction error
round(nmf.reconstruction_err_, 2)
# The error can be interpreted relative to the error of other models

414.49

# Reconstruct the original matrix

In [40]:
# Look at the original matrix R
R.head()

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# Calculate R_hat
R_hat = pd.DataFrame(np.matmul(P, Q), index=R.index, columns=R.columns)
R_hat

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
userId,,,,,,,,,,,,,,,,,,,,,
1,4.262075,3.765553,3.838771,2.641187,3.613020,4.463453,3.454617,2.925595,3.245245,3.865596,...,3.499959,2.999965,3.999953,3.999953,3.499959,3.999953,3.499959,3.499959,3.499959,3.999953
2,3.902953,3.428861,3.407542,2.339483,3.074723,3.962653,3.209069,2.771865,2.800618,3.659109,...,3.498242,2.998493,3.997991,3.997991,3.498242,3.997991,3.498242,3.498242,3.498242,3.997991
3,3.574294,2.855586,2.484219,1.911068,2.507400,3.805785,2.811172,2.578403,2.469384,3.544649,...,3.505394,3.004623,4.006164,4.006164,3.505394,4.006164,3.505394,3.505394,3.505394,4.006164
4,4.353018,3.565942,3.316833,2.275702,3.139630,3.667827,3.109239,2.656514,3.557203,3.282599,...,3.500721,3.000618,4.000824,4.000824,3.500721,4.000824,3.500721,3.500721,3.500721,4.000824
5,4.057221,3.307349,2.942012,2.332712,2.882049,4.036015,3.128256,2.634574,2.543501,3.511562,...,3.505103,3.004374,4.005832,4.005832,3.505103,4.005832,3.505103,3.505103,3.505103,4.005832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,4.048159,3.423570,3.101784,2.537292,2.645506,3.907733,3.228592,2.526416,2.711616,3.263002,...,3.498750,2.998928,3.998571,3.998571,3.498750,3.998571,3.498750,3.498750,3.498750,3.998571
607,3.626363,3.602978,3.932596,2.788803,3.347047,4.126821,3.562724,2.846453,3.336610,3.496924,...,3.503037,3.002603,4.003471,4.003471,3.503037,4.003471,3.503037,3.503037,3.503037,4.003471
608,2.742833,2.882308,2.733418,2.202079,2.291152,3.818929,2.834102,2.538355,2.703833,3.541894,...,3.498506,2.998720,3.998293,3.998293,3.498506,3.998293,3.498506,3.498506,3.498506,3.998293


# Make prediction based on new user input

In [43]:
len(R.index)

610

In [44]:
# Create a dictionary for a new user
new_user_input = {'Silver Spoon (2014)':3.2, 'Flint (2017)':4.6, 'Jumanji (1995)':5.0} # similar to JSON data that we will have to work with in the end
new_user_input

{'Silver Spoon (2014)': 3.2, 'Flint (2017)': 4.6, 'Jumanji (1995)': 5.0}

In [45]:
# Convert it to a pd.DataFrame
new_user = pd.DataFrame(new_user_input, columns=R.columns, index=[len(R.index)+1])
new_user

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
611,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.2,NaN,NaN,NaN,NaN,4.6,NaN,NaN


In [46]:
#Fill missing data
new_user = new_user.fillna(average_movie_rating)
new_user

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
611,3.262448,5.0,3.262448,3.262448,3.262448,3.262448,3.262448,3.262448,3.262448,3.262448,...,3.262448,3.262448,3.2,3.262448,3.262448,3.262448,3.262448,4.6,3.262448,3.262448


In [48]:
#Prediction step 1 - generate user_P 
user_P = nmf.transform(new_user)
user_P

array([[2.27368201, 0.00749295, 0.16676883, 0.23732373, 0.23527507,
        1.1408119 , 0.04834332, 0.18136098, 0.09784989, 0.1776735 ,
        0.17833698, 0.25115823, 0.12395009, 0.44014572, 0.        ,
        2.29387523, 0.        , 0.        , 1.15972838, 0.11672222]])

In [49]:
#new user R - reconstruct R but for this new user only
user_R = pd.DataFrame(np.matmul(user_P, Q), index=new_user.index, columns=R.columns)
user_R

title,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),...,Gintama: The Movie (2010),anohana: The Flower We Saw That Day - The Movie (2013),Silver Spoon (2014),Love Live! The School Idol Movie (2015),Jon Stewart Has Left the Building (2015),Black Butler: Book of the Atlantic (2017),No Game No Life: Zero (2017),Flint (2017),Bungo Stray Dogs: Dead Apple (2018),Andrew Dice Clay: Dice Rules (1991)
611,3.675764,3.48419,3.613799,2.383092,2.983588,3.66843,2.937971,2.929032,3.388021,3.724634,...,3.229621,2.768247,3.690995,3.690995,3.229621,3.690995,3.229621,3.229621,3.229621,3.690995


In [50]:
# I have a list of predicted films!! Can I now use this for my recommendations?
# We want to get rid of movies we have already watchend
recommendation = user_R.drop(columns=new_user_input.keys())

In [51]:
# Sort recommendations
recommendation.sort_values(by=len(R.index)+1, axis=1, ascending=False, inplace=True)

/Users/anja.jesierski/opt/anaconda3/envs/dl/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [52]:
round(recommendation, 2)

title,"Sandpiper, The (1965)",Tokyo Tribe (2014),George Carlin: Back in Town (1996),George Carlin: You Are All Diseased (1999),Larry David: Curb Your Enthusiasm (1999),Tom and Jerry: Shiver Me Whiskers (2006),Kung Fu Panda: Secrets of the Masters (2011),The Eye: Infinity (2005),Villain (1971),"Decalogue, The (Dekalog) (1989)",...,Alone in the Dark (2005),"Haunted House 2, A (2014)",Aloha (2015),Are We There Yet? (2005),Sorrow (2015),"Spy Who Came in from the Cold, The (1965)",Journey 2: The Mysterious Island (2012),God's Not Dead (2014),Survivor (2015),Carnival Magic (1981)
611,4.61,4.61,4.61,4.61,4.61,4.61,4.61,4.61,4.61,4.61,...,0.46,0.46,0.46,0.46,0.46,0.46,0.46,0.46,0.46,0.46


In [53]:
round(recommendation.iloc[:,0:5], 2)

title,"Sandpiper, The (1965)",Tokyo Tribe (2014),George Carlin: Back in Town (1996),George Carlin: You Are All Diseased (1999),Larry David: Curb Your Enthusiasm (1999)
611,4.61,4.61,4.61,4.61,4.61
